In [ ]:
import spacy
import pandas as pd
import os
from params.configures import Config_path, Config_output_path

In [ ]:
## locate parent directory
# It suggest to train models on Google Colab GPU (connect to your own google drive)
config = Config_path()
os.chdir(config.parent_path)


In [ ]:
#########################
## Load trained models ##
#########################
NER = spacy.load("./oneHop_QA/NER_output_train-dev/model-best") #load the best model
CATS = spacy.load("./oneHop_QA/cat_output_train-train_268_0719/model-best") #load the best model

In [ ]:
question = "which genre of album is harder faster"
doc = NER(question) # input sample text
print(doc.ents[0])
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter Notebook/Colab

In [ ]:
#####################
## Display Metrics ##
#####################
from oneHop_QA.metrics import test_pipe,mask_doc,NER_metrics
predict_cats = test_pipe(question,NER,CATS)
print('relation prediction:',predict_cats)

In [ ]:
c = Config_path()
test = pd.read_csv(c.test_combination_path)
test_q = test['question'].values
test_ents = test['entity'].values
precision, recall, f1= NER_metrics(test_q[0:1000],test_ents[0:1000],NER)
print('Named Entity Metrics',precision, recall, f1)

In [ ]:
from sklearn.metrics import f1_score

test_q_space = test['q_space'].values
test_r = test['relationship'].values
test_r = [i.rsplit('/', 2)[0:-2] for i in test_r]

redict_r = []
for i in test_q_space[0:1000]:
  doc_cats = CATS(i)
  pedict_relation = max(doc_cats.cats, key=doc_cats.cats.get)
  redict_r.append(pedict_relation)

In [ ]:
f1_CATS = f1_score(test_r[0:1000], redict_r, average='weighted')
print('relation prediction f1 score:',f1_CATS)